In [35]:
!ls 

configs  filelists	       logs	   resyntesis
dataset  inference_test.ipynb  resynt_cpy  resyntesis_meld.ipynb


In [2]:
import so_vits_svc_fork
dir(so_vits_svc_fork)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'init_logger',
 'logger']

In [3]:
from so_vits_svc_fork.inference.main import infer

In [19]:
help(infer)

Help on function infer in module so_vits_svc_fork.inference.main:

infer(*, input_path: 'Path | str | Sequence[Path | str]', output_path: 'Path | str | Sequence[Path | str]', model_path: 'Path | str', config_path: 'Path | str', recursive: 'bool' = False, speaker: 'int | str', cluster_model_path: 'Path | str | None' = None, transpose: 'int' = 0, auto_predict_f0: 'bool' = False, cluster_infer_ratio: 'float' = 0, noise_scale: 'float' = 0.4, f0_method: "Literal[('crepe', 'crepe-tiny', 'parselmouth', 'dio', 'harvest')]" = 'dio', db_thresh: 'int' = -40, pad_seconds: 'float' = 0.5, chunk_seconds: 'float' = 0.5, absolute_thresh: 'bool' = False, max_chunk_seconds: 'float' = 40, device: 'str | torch.device' = device(type='cuda', index=0))



In [24]:
import pandas as pd

df = pd.read_csv('../../sambashare/MELD/MELD_audio/train_sent_emo.csv')

speaker_list = ['Chandler', 'Joey', 'Monica', 'Phoebe', 'Rachel', 'Ross']
speaker_dict = {}
for i, spk in enumerate(df.Speaker.values):
    if spk in speaker_list:
        if spk not in speaker_dict:
            speaker_dict[spk] = []
        dia_id, utt_id = df.loc[i, ['Dialogue_ID', 'Utterance_ID']]
        audio = '../../sambashare/MELD/MELD_audio/train_audio/' + f'dia{dia_id}_utt{utt_id}.mp3'
        speaker_dict[spk].append(audio)

speaker_dict.keys()

dict_keys(['Chandler', 'Joey', 'Rachel', 'Monica', 'Phoebe', 'Ross'])

In [87]:
import IPython.display as ipd
ipd.Audio('../../sambashare/MELD/MELD_audio/train_audio/dia0_utt0.mp3')

In [36]:
!mkdir train_resyntesis

In [38]:
!ls

configs    inference_test.ipynb  resyntesis
dataset    logs			 resyntesis_meld.ipynb
filelists  resynt_cpy		 train_resyntesis


In [47]:
import os
import random 

In [116]:
%%time 
pretrain_models = ['andre-yaniv-so-vits-svc', 'jokowi-so-vits-svc-model', 
                   'Lain-so-vits-svc-4.0', 'so-vits-svc-4.0-danxiao',
                    'so-vits-svc-genshinn', 'tamao_sovits']

batch_size = 1024
for spk in speaker_dict:
    for batch_id in range(len(speaker_dict[spk]) // batch_size):
        batch = speaker_dict[spk][batch_id * batch_size: batch_id * batch_size + batch_size - 1]
        
        target_spk = random.choice(pretrain_models)
        
        outputs = [f'train_resyntesis/{target_spk}/{sample.split("/")[-1]}' for sample in batch]
        
        model = sorted([file for file in os.listdir(f'pretrain_models/{target_spk}') 
                        if file[-3:] == 'pth' and file[0] == 'G'], 
                        key=lambda x: -int(x.split('.')[0].split('_')[-1]))[0]

        # TODO: model re-dif infer-model
        infer(input_path=batch, 
              output_path=outputs, 
              model_path=f'pretrain_models/{target_spk}/{model}', 
              config_path=f'pretrain_models/{target_spk}/config.json',
              speaker=None,
              f0_method='dio',
              auto_predict_f0=True,
             );

        
# infer("tests/dataset_raw/34j/1.wav", "tests/configs/config.json", "tests/logs/44k")

/mnt/storage/kocharyan/so-vits-svc-fork/svc_proj/lib/python3.8/site-packages/so_vits_svc_fork/modules/synthesizers.py:81: UserWarning: Unused arguments: {'n_layers_q': 3, 'use_spectral_norm': False}
  warnings.warn(f"Unused arguments: {kwargs}")
[22:02:02] Decoder type: hifi-gan
[22:02:04] Loaded checkpoint 'pretrain_models/andre-yaniv-so-vits-svc/G_15000.pth' (epoch 15000)
../../sambashare/MELD/MELD_audio/train_audio/dia0_utt0.mp3:   0%|                              | 0/1023 [00:00<?, ?it/s][22:02:04] Chunk: Chunk(Speech: True, 250928.0)
[22:02:04] Speaker None is not found. Use speaker 0 instead.
[22:02:04] F0 inference time:       0.208s, RTF: 0.031
/mnt/storage/kocharyan/so-vits-svc-fork/svc_proj/lib/python3.8/site-packages/so_vits_svc_fork/utils.py:179: UserWarning: legacy_final_proj is deprecated
  warnings.warn("legacy_final_proj is deprecated")
[22:02:04] HuBERT inference time  : 0.009s, RTF: 0.001
[22:02:04] Inferece time: 0.04s, RTF: 0.01
../../sambashare/MELD/MELD_audio/trai

../../sambashare/MELD/MELD_audio/train_audio/dia6_utt14.mp3:   2%|▍                   | 20/1023 [00:06<06:46,  2.47it/s][22:02:10] Chunk: Chunk(Speech: True, 26460.0)
[22:02:10] Speaker None is not found. Use speaker 0 instead.
[22:02:10] F0 inference time:       0.045s, RTF: 0.028
[22:02:11] HuBERT inference time  : 0.010s, RTF: 0.006
[22:02:11] Inferece time: 0.05s, RTF: 0.03
../../sambashare/MELD/MELD_audio/train_audio/dia6_utt15.mp3:   2%|▍                   | 21/1023 [00:06<05:40,  2.94it/s][22:02:11] Chunk: Chunk(Speech: True, 31752.0)
[22:02:11] Speaker None is not found. Use speaker 0 instead.
[22:02:11] F0 inference time:       0.044s, RTF: 0.026
[22:02:11] HuBERT inference time  : 0.010s, RTF: 0.006
[22:02:11] Inferece time: 0.05s, RTF: 0.03
../../sambashare/MELD/MELD_audio/train_audio/dia6_utt16.mp3:   2%|▍                   | 22/1023 [00:07<04:58,  3.36it/s][22:02:11] Chunk: Chunk(Speech: True, 57771.0)
[22:02:11] Speaker None is not found. Use speaker 0 instead.
[22:02:11]

[22:02:17] F0 inference time:       0.051s, RTF: 0.022
[22:02:17] HuBERT inference time  : 0.010s, RTF: 0.005
[22:02:17] Inferece time: 0.05s, RTF: 0.02
../../sambashare/MELD/MELD_audio/train_audio/dia12_utt8.mp3:   4%|▊                   | 42/1023 [00:13<05:34,  2.93it/s][22:02:17] Chunk: Chunk(Speech: True, 133182.0)
[22:02:17] Speaker None is not found. Use speaker 0 instead.
[22:02:18] F0 inference time:       0.091s, RTF: 0.023
[22:02:18] HuBERT inference time  : 0.011s, RTF: 0.003
[22:02:18] Inferece time: 0.05s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia12_utt10.mp3:   4%|▊                  | 43/1023 [00:13<05:36,  2.92it/s][22:02:18] Chunk: Chunk(Speech: True, 52479.0)
[22:02:18] Speaker None is not found. Use speaker 0 instead.
[22:02:18] F0 inference time:       0.045s, RTF: 0.021
[22:02:18] HuBERT inference time  : 0.010s, RTF: 0.005
[22:02:18] Inferece time: 0.05s, RTF: 0.02
../../sambashare/MELD/MELD_audio/train_audio/dia17_utt2.mp3:   4%|▊                 

../../sambashare/MELD/MELD_audio/train_audio/dia40_utt10.mp3:   6%|█▏                 | 63/1023 [00:21<03:55,  4.07it/s][22:02:25] Chunk: Chunk(Speech: True, 83349.0)
[22:02:25] Speaker None is not found. Use speaker 0 instead.
[22:02:25] F0 inference time:       0.050s, RTF: 0.017
[22:02:26] HuBERT inference time  : 0.009s, RTF: 0.003
[22:02:26] Inferece time: 0.04s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia42_utt0.mp3:   6%|█▎                  | 64/1023 [00:21<03:54,  4.09it/s][22:02:26] Chunk: Chunk(Speech: True, 245637.0)
[22:02:26] Speaker None is not found. Use speaker 0 instead.
[22:02:26] F0 inference time:       0.178s, RTF: 0.027
[22:02:26] HuBERT inference time  : 0.009s, RTF: 0.001
[22:02:26] Inferece time: 0.05s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia45_utt1.mp3:   6%|█▎                  | 65/1023 [00:22<05:21,  2.98it/s][22:02:26] Chunk: Chunk(Speech: True, 133182.0)
[22:02:26] Speaker None is not found. Use speaker 0 instead.
[22:02:2

[22:02:34] F0 inference time:       0.087s, RTF: 0.028
[22:02:34] HuBERT inference time  : 0.008s, RTF: 0.003
[22:02:34] Inferece time: 0.04s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia57_utt2.mp3:   8%|█▋                  | 85/1023 [00:30<07:47,  2.01it/s][22:02:34] Chunk: Chunk(Speech: True, 332073.0)
[22:02:34] Speaker None is not found. Use speaker 0 instead.
[22:02:35] F0 inference time:       0.208s, RTF: 0.024
[22:02:35] HuBERT inference time  : 0.008s, RTF: 0.001
[22:02:35] Inferece time: 0.05s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia57_utt3.mp3:   8%|█▋                  | 86/1023 [00:31<08:42,  1.79it/s][22:02:35] Chunk: Chunk(Speech: True, 258426.0)
[22:02:35] Speaker None is not found. Use speaker 0 instead.
[22:02:35] F0 inference time:       0.188s, RTF: 0.027
[22:02:35] HuBERT inference time  : 0.008s, RTF: 0.001
[22:02:35] Inferece time: 0.04s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia61_utt5.mp3:   9%|█▋               

...

[22:40:56] F0 inference time:       0.102s, RTF: 0.030
[22:40:56] HuBERT inference time  : 0.008s, RTF: 0.002
[22:40:56] Inferece time: 0.03s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia681_utt5.mp3:  93%|████████████████▋ | 947/1023 [06:15<00:29,  2.55it/s][22:40:56] Chunk: Chunk(Speech: True, 46746.0)
[22:40:56] Speaker None is not found. Use speaker 0 instead.
[22:40:57] F0 inference time:       0.048s, RTF: 0.023
[22:40:57] HuBERT inference time  : 0.008s, RTF: 0.004
[22:40:57] Inferece time: 0.04s, RTF: 0.02
../../sambashare/MELD/MELD_audio/train_audio/dia682_utt0.mp3:  93%|████████████████▋ | 948/1023 [06:15<00:25,  2.99it/s][22:40:57] Chunk: Chunk(Speech: True, 352359.0)
[22:40:57] Speaker None is not found. Use speaker 0 instead.
[22:40:57] F0 inference time:       0.206s, RTF: 0.023
[22:40:57] HuBERT inference time  : 0.011s, RTF: 0.001
[22:40:57] Inferece time: 0.04s, RTF: 0.00
../../sambashare/MELD/MELD_audio/train_audio/dia682_utt1.mp3:  93%|████████████████▋

../../sambashare/MELD/MELD_audio/train_audio/dia700_utt7.mp3:  95%|█████████████████ | 968/1023 [06:24<00:32,  1.67it/s][22:41:06] Chunk: Chunk(Speech: True, 53361.0)
[22:41:06] Speaker None is not found. Use speaker 0 instead.
[22:41:06] F0 inference time:       0.054s, RTF: 0.024
[22:41:06] HuBERT inference time  : 0.010s, RTF: 0.005
[22:41:06] Inferece time: 0.04s, RTF: 0.02
../../sambashare/MELD/MELD_audio/train_audio/dia700_utt9.mp3:  95%|█████████████████ | 969/1023 [06:25<00:26,  2.04it/s][22:41:06] Chunk: Chunk(Speech: True, 109368.0)
[22:41:06] Speaker None is not found. Use speaker 0 instead.
[22:41:06] F0 inference time:       0.086s, RTF: 0.025
[22:41:06] HuBERT inference time  : 0.011s, RTF: 0.003
[22:41:06] Inferece time: 0.04s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia700_utt11.mp3:  95%|████████████████ | 970/1023 [06:25<00:27,  1.95it/s][22:41:06] Chunk: Chunk(Speech: True, 107604.0)
[22:41:06] Speaker None is not found. Use speaker 0 instead.
[22:41:0

[22:41:13] F0 inference time:       0.049s, RTF: 0.019
[22:41:13] HuBERT inference time  : 0.010s, RTF: 0.004
[22:41:13] Inferece time: 0.04s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia726_utt3.mp3:  97%|█████████████████▍| 990/1023 [06:32<00:09,  3.48it/s][22:41:13] Chunk: Chunk(Speech: True, 71001.0)
[22:41:13] Speaker None is not found. Use speaker 0 instead.
[22:41:13] F0 inference time:       0.048s, RTF: 0.018
[22:41:13] HuBERT inference time  : 0.011s, RTF: 0.004
[22:41:14] Inferece time: 0.08s, RTF: 0.03
../../sambashare/MELD/MELD_audio/train_audio/dia726_utt5.mp3:  97%|█████████████████▍| 991/1023 [06:33<00:09,  3.47it/s][22:41:14] Chunk: Chunk(Speech: True, 228879.0)
[22:41:14] Speaker None is not found. Use speaker 0 instead.
[22:41:14] F0 inference time:       0.170s, RTF: 0.027
[22:41:14] HuBERT inference time  : 0.011s, RTF: 0.002
[22:41:14] Inferece time: 0.04s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia726_utt7.mp3:  97%|█████████████████

../../sambashare/MELD/MELD_audio/train_audio/dia735_utt2.mp3:  99%|████████████████▊| 1011/1023 [06:40<00:04,  2.65it/s][22:41:22] Chunk: Chunk(Speech: True, 405719.0)
[22:41:22] Speaker None is not found. Use speaker 0 instead.
[22:41:22] F0 inference time:       0.190s, RTF: 0.019
[22:41:22] HuBERT inference time  : 0.011s, RTF: 0.001
[22:41:22] Inferece time: 0.04s, RTF: 0.00
../../sambashare/MELD/MELD_audio/train_audio/dia735_utt4.mp3:  99%|████████████████▊| 1012/1023 [06:41<00:05,  2.03it/s][22:41:22] Chunk: Chunk(Speech: True, 197568.0)
[22:41:22] Speaker None is not found. Use speaker 0 instead.
[22:41:22] F0 inference time:       0.097s, RTF: 0.018
[22:41:22] HuBERT inference time  : 0.011s, RTF: 0.002
[22:41:23] Inferece time: 0.04s, RTF: 0.01
../../sambashare/MELD/MELD_audio/train_audio/dia735_utt6.mp3:  99%|████████████████▊| 1013/1023 [06:42<00:04,  2.11it/s][22:41:23] Chunk: Chunk(Speech: True, 127919.0)
[22:41:23] Speaker None is not found. Use speaker 0 instead.
[22:41:

CPU times: user 42min 44s, sys: 3min 3s, total: 45min 48s
Wall time: 39min 26s


In [115]:

# for spk in speaker_dict:
#     for batch_id in range(len(speaker_dict[spk]) // batch_size):
#         print()

In [109]:
# !ls pretrain_models/tamao_sovits/

In [110]:
# infer(input_path='../../sambashare/MELD/MELD_audio/train_audio/dia0_utt0.mp3', 
#               output_path='file4.mp3', 
#               model_path=f'pretrain_models/tamao_sovits/G_67.pth', 
#               config_path=f'pretrain_models/tamao_sovits/config.json',
#               speaker=None,
#               f0_method='dio',
#               auto_predict_f0=True
#              )

In [111]:
# help(infer())

In [112]:
# ! svc infer -h

In [113]:
# os.system('svc infer ../../sambashare/MELD/MELD_audio/train_audio/dia0_utt0.mp3 -m pretrain_models/tamao_sovits/G_67.pth -c pretrain_models/tamao_sovits/config.json -o file.mp3')